In [2]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [4]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [5]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [6]:
def chat(message,history):
    messages =[{"role":"system","content":system_message}]
    for human, assistant in history:
        messages.append({"role":"user","content":human})
        messages.appenf({"role":"assistant","content":assistant})
    messages.append({"role":"user","content":message})
    respose =openai.chat.completions.create(model=MODEL,messages=messages)
    return response.choices[0].message.content


gr.ChatInterface(fn=chat).launch()
    

D:\gen-ai\anaconda\envs\llms\Lib\site-packages\gradio\chat_interface.py:348: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [7]:
ticket_prices ={"usa":"$100","london":"$90","australia":"$150","tokyo":"$180"}
def get_ticket_price(destination):
    
    city =destination.lower()
    print(f"the ticket price for {destination} is : ")
    return ticket_prices.get(city,"unknown")

In [8]:
get_ticket_price("usa")

the ticket price for usa is : 


'$100'

In [9]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination"],
        "additionalProperties": False
    }
}

In [10]:
tools =[{"type":"function","function":price_function}]

In [11]:
def chat(message,history):
    messages =[{"role":"system","content":system_message}] +history+[{"role":"user","content":message}]
    response =openai.chat.completions.create(model=MODEL,messages =messages,tools=tools)

    if response.choices[0].finish_reason=='tool_calls':
        message =response.choices[0].message
        response,city =handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response =openai.chat.completions.create(model=MODEL,messages=messages)
    return response.choices[0].message.content

In [12]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city
    

In [13]:
!pip install intel-gradio-theme


In [25]:
!pip install aina-gradio-theme


In [31]:

import gradio as gr

import AinaTheme

gr.ChatInterface(fn=chat,type="messages",title="AIRLINE AI ASSISTANT",description="ASK ANYTHING ABOUT YOUR JOURNEY",theme="soft").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


the ticket price for London is : 
the ticket price for Australia is : 


In [ ]:
def artist(city):
    image_response=openai.images.generate